In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

## Q1

In [2]:
df = pd.read_parquet("data/fhv_tripdata_2021-01.parquet")
df.shape

(1154112, 7)

## Q2

In [3]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [4]:
df.duration.mean()

19.167224093791006

## Q3

In [5]:
df_drop = df[(df.duration >= 1) & (df.duration <= 60)]

In [6]:
print(df_drop.shape, df.shape)
print("Dropped: ", df.shape[0] - df_drop.shape[0])

(1109826, 8) (1154112, 8)
Dropped:  44286


In [7]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['trip_distance']
df_dropc = df_drop[categorical].fillna(-1).astype(int).astype(str)

In [8]:
(df_dropc['PUlocationID'] == "-1").sum() / df_dropc['PUlocationID'].shape[0] * 100

83.52732770722618

## Q4

In [9]:
train_dicts = df_dropc[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(1109826, 525)

## Q5

In [10]:
target = 'duration'
y_train = df_drop[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [12]:
y_pred = lr.predict(X_train)
mean_squared_error(y_pred, y_train, squared=False)

10.528519107212675

## Q6

In [13]:
dff = pd.read_parquet("data/fhv_tripdata_2021-02.parquet")
dff['duration'] = dff.dropOff_datetime - dff.pickup_datetime
dff.duration = dff.duration.apply(lambda td: td.total_seconds() / 60)
dff_drop = dff[(dff.duration >= 1) & (dff.duration <= 60)]
dff_dropc = dff_drop[categorical].fillna(-1).astype(int).astype(str)
val_dicts = dff_dropc[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)
y_val = dff_drop[target].values

In [14]:
y_val_pred = lr.predict(X_val)
mean_squared_error(y_val_pred, y_val, squared=False)

11.014283191827879